In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures
import time

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [2]:
##bing
start = time.perf_counter()
def get_info(stock):
    temp_frame=pd.DataFrame()
    tick = yf.Ticker(stock)
    tick_hist = tick.history(start=start_date,end=end_date)
    if len(tick_hist.index) != 0:
        tick_info = tick.info
        if tick_info['currency'] == 'USD':
            if tick_hist['Volume'].mean()>= 10000:
                for need in hist_need:
                    temp_frame[need+' of '+stock]=tick_hist[need]
                return (temp_frame,stock, tick_info['sector'])
    return pd.DataFrame(),'',''
start_date = '2021-07-02'
end_date = '2021-10-22'
hist_data = pd.DataFrame()
clean_stocks = []
industry_list = []
hist_need = ['Open','High','Low','Close','Volume']
tick_data = pd.read_csv("Tickers.csv",names=["Tickers"])
#with concurrent.futures.ThreadPoolExecutor() as executor:
list_tick = []
for x in range(len(tick_data.index)):
    list_tick.append(tick_data['Tickers'].iloc[x])
print(list_tick)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_info, list_tick)
    for result in results:
        hist_data = pd.concat([hist_data,result[0]],join='outer',axis=1)
        clean_stocks.append(result[1])
        industry_list.append(result[2])
while True:
    if "" not in clean_stocks:
        break
    else:
        industry_list.remove("")
        clean_stocks.remove("")
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds(s)')
hist_data.dropna(inplace=True)
print(clean_stocks)
hist_data.head()

['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PCLN', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'SHOP.TO', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TD.TO', 'TGT', 'TWX', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']
- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
Finished in 26.61 seconds(s)
['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL',

,Open of AAPL,High of AAPL,Low of AAPL,Close of AAPL,Volume of AAPL,Open of ABBV,High of ABBV,Low of ABBV,Close of ABBV,Volume of ABBV,...,Open of UPS,High of UPS,Low of UPS,Close of UPS,Volume of UPS,Open of USB,High of USB,Low of USB,Close of USB,Volume of USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-02,137.493033,139.586842,137.343482,139.546967,78852600.0,111.900746,112.848562,111.285158,112.535881,5719500.0,...,207.463364,209.947906,207.156512,209.383682,2172300.0,57.171395,57.300406,56.744668,57.131699,3520900.0
2021-07-06,139.656634,142.727531,139.656634,141.600876,108181800.0,112.535879,113.249187,112.145027,113.083076,7172100.0,...,208.324553,209.462879,206.077582,209.146118,2421300.0,56.665274,56.893526,55.206468,55.583572,5008700.0
2021-07-07,143.116382,144.462404,142.238989,144.143356,104911600.0,113.258966,114.294720,112.672683,114.079750,6695100.0,...,209.165919,212.749183,208.185974,212.085983,2553700.0,55.246161,56.049996,55.037761,55.563728,5125700.0
2021-07-08,141.162171,143.634848,140.254853,142.817276,105575500.0,113.112394,114.138374,113.034222,113.688896,3948800.0,...,208.185954,210.541795,207.770203,209.918198,2354300.0,54.700349,55.216391,54.253774,54.620960,5977700.0
2021-07-09,142.328727,145.220162,142.229016,144.681763,99890800.0,113.640040,114.646482,113.454384,113.913635,4823400.0,...,211.719726,212.323537,210.512104,211.749420,2050500.0,55.653043,56.605733,55.424791,56.536266,4437300.0


In [3]:
def get_daily_returns(list_of_stock):
    monthly_returns = pd.DataFrame()
    for stock in list_of_stock:
        monthly_returns['Monthly Returns of '+stock] = hist_data['Close of '+stock].pct_change()*100
    monthly_returns.dropna(inplace=True)
    return monthly_returns
def get_beta_total(hello):
    tot = 0
    for x in range (len(hello)):
        tot+= beta_vals['Monthly Returns of '+hello[x]]
    return (tot/len(hello)).iloc[0]
def get_average_std(bye):
    tot = 0
    for x in range (len(bye)):
        tot+= monthly_returns['Monthly Returns of '+bye[x]].std(axis=0,skipna=True)
    return (tot/len(bye))
def get_alpha(stocks_lst):
    # Get the current beta column
    column_counter = 1

    alpha = pd.DataFrame()
    for stock in stocks_lst:
        alpha['Alpha '+stock] = monthly_returns['Monthly Returns of '+stock] - ((monthly_returns['Monthly Returns of '+stock].mean()) * beta_vals.iloc[0, column_counter])
        column_counter += 1

    alpha.head()

    # Calculate the mean of the alpha

 #   for stock in stocks_lst:
  #      alpha_mean.append(alpha['Alpha ' + stock].mean()) 

   # positive_counter = 0
    #negative_counter = 0

 #   for i in range(len(alpha_mean)):
  #      if(alpha_mean[i] >= 0):
   #         positive_counter += 1
#
 #       else:
  #          negative_counter += 1
    return alpha
# MDD Equation: (Peak Value - Trough Value) / Peak Value
def get_mdd(stocks):

    mdd = []

    for x in range (len(stocks)):

        trough = min(hist_data['Low of '+stocks[x]])
        peak = max(hist_data['High of '+stocks[x]])

        mdd.append((((peak - trough) / peak),stocks[x]))

    return mdd

In [6]:
monthly_returns = get_daily_returns(clean_stocks)
monthly_returns.head()

,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,Monthly Returns of BIIB,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-06,1.471841,0.486241,0.660795,0.183700,-2.996878,4.692707,0.629081,-0.228153,-2.624553,-1.098299,...,-4.940534,0.178772,-1.535144,-0.718435,-0.458268,-0.525472,0.219863,-0.404951,-0.113459,-2.709750
2021-07-07,1.795526,0.881364,0.883694,1.378522,0.386187,0.566963,0.837459,-1.846363,-0.798599,3.027044,...,-1.572021,0.081111,-1.449386,-0.310134,0.855572,-0.742682,0.341241,2.028508,1.405651,-0.035701
2021-07-08,-0.919973,-0.342615,-0.508889,-0.148580,-4.124817,0.942218,-2.035319,2.152906,-2.440249,3.861200,...,-0.488914,-0.226939,-0.556491,-0.809573,0.416077,-0.811455,-0.415389,-4.383626,-1.022126,-1.696733
2021-07-09,1.305505,0.197679,0.402476,1.122430,4.480600,-0.323466,2.650744,1.191026,3.249103,-2.950815,...,2.161804,-0.129972,3.677358,0.958130,0.000000,1.078407,0.526891,1.534306,0.872351,3.506540
2021-07-12,-0.420381,0.900663,-0.776679,-0.335861,1.216133,-0.021241,0.965453,-0.542595,1.473522,-2.546347,...,0.096184,0.536845,0.447218,0.105440,1.472372,0.867193,0.953630,-0.212007,-0.275798,1.070743


In [7]:
beta_data = pd.DataFrame()
MarketIndex='^GSPTSE'
marketing_tick = yf.Ticker(MarketIndex)
marketing_hist = marketing_tick.history(start=start_date,end=end_date)
beta_data[MarketIndex] = marketing_hist['Close'].pct_change()*100
beta_data = pd.concat([beta_data, monthly_returns],join='inner',axis=1)
MarketVar= beta_data['^GSPTSE'].var()
Beta = pd.DataFrame(beta_data.cov() / MarketVar)
beta_vals = pd.DataFrame(Beta.iloc[[0]])
beta_vals.rename(index={'^GSPTSE': 'Beta'})
beta_vals.head()

,^GSPTSE,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
^GSPTSE,1.0,1.30269,0.413482,0.364003,0.995564,2.092086,1.032426,1.727356,1.858664,1.883553,...,2.428232,0.212841,1.949599,0.733672,1.014328,1.173329,1.058612,1.254119,0.669891,1.837344


In [8]:
start = time.perf_counter()
tech_list = []
health_list = []
industrial_list = []
finance_list = []
defensive_list = []
cyclical_list = []
real_estate_list = []
energy_list = []
materials_list = []
communication_list = []
utilities_list = []

for (industry,stock) in zip(industry_list,clean_stocks):
    if industry == "Technology":
        tech_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Healthcare":
        health_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Industrials":
        industrial_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Financial Services":
        finance_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Consumer Defensive":
        defensive_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Consumer Cyclical":
        cyclical_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Real Estate":
        real_estate_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Energy":
        energy_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Basic Materials":
        materials_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Communication Services":
        communication_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Utilities":
        utilities_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
list_of_industries = [tech_list,health_list,industrial_list,finance_list,defensive_list,cyclical_list,real_estate_list,energy_list,materials_list,communication_list,utilities_list]

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds(s)')

Finished in 0.0 seconds(s)


In [9]:
tech_list.sort()
health_list.sort()
industrial_list.sort()
finance_list.sort()
defensive_list.sort()
cyclical_list.sort()
real_estate_list.sort()
energy_list.sort()
materials_list.sort()
communication_list.sort()
utilities_list.sort()

In [10]:
total_list = []
if len(tech_list)!=0:
    total_list.append(tech_list[0][1])
    
if len(health_list)!=0:
    total_list.append(health_list[0][1])
    
if len(industrial_list)!=0:
    total_list.append(industrial_list[0][1])
    
if len(finance_list)!=0:
    total_list.append(finance_list[0][1])
    
if len(defensive_list)!=0:
    total_list.append(defensive_list[0][1])
    
if len(cyclical_list)!=0:
    total_list.append(cyclical_list[0][1])
    
if len(real_estate_list)!=0:
    total_list.append(real_estate_list[0][1])
    
if len(energy_list)!=0:
    total_list.append(energy_list[0][1])
    
if len(materials_list)!=0:
    total_list.append(materials_list[0][1])
    
if len(communication_list)!=0:
    total_list.append(communication_list[0][1])
    
if len(utilities_list)!=0:
    total_list.append(utilities_list[0][1])
    

In [11]:
total_list 

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO']

In [12]:
alpha_mean = []
alpha = get_alpha(total_list)
alpha.head()

,Alpha CSCO,Alpha BMY,Alpha LMT,Alpha MON,Alpha PG,Alpha SBUX,Alpha SPG,Alpha KMI,Alpha T,Alpha SO
Date,,,,,,,,,,
2021-07-06,-1.135069,-0.371743,-0.733433,0.762531,-0.054235,0.654463,-1.839876,-0.616698,-0.496046,0.178476
2021-07-07,0.439370,0.811527,1.275998,-0.014267,0.637012,1.211754,-1.754118,-0.675869,-0.087746,0.080816
2021-07-08,-0.089126,-0.593890,-0.922573,0.181006,-0.127706,-0.988317,-0.861223,-0.627016,-0.587185,-0.227235
2021-07-09,0.812122,0.781099,0.845405,-0.896403,-0.076603,1.269391,3.372626,2.835160,1.180519,-0.130268
2021-07-12,-1.038144,0.582584,-1.048754,0.368634,-0.032825,0.836169,0.142485,-0.720083,0.327828,0.536550


In [13]:

mdd = get_mdd(total_list)
mdd

[(0.1315745272407484, 'CSCO'),
 (0.18105057874970099, 'BMY'),
 (0.12263098504021946, 'LMT'),
 (0.034170870261855966, 'MON'),
 (0.08636826787558043, 'PG'),
 (0.1305045609406697, 'SBUX'),
 (0.21786154608093122, 'SPG'),
 (0.17537307844185201, 'KMI'),
 (0.1154308691397112, 'T'),
 (0.10941946845955973, 'SO')]

In [14]:
for x in range (len(total_list)):
    clean_stocks.remove(total_list[x])

In [15]:
total_list

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO']

In [16]:
list_of_beta = []
for stock in clean_stocks:
    list_of_beta.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
list_of_beta.sort()
list_of_beta

[(0.7688321597213277, 0.4999981723218368, 'KO'),
 (0.8070678497721255, 0.27208094687927487, 'PEP'),
 (0.9473982893659325, 0.1479497575862508, 'CL'),
 (0.9523077719862846, 0.5814648713566358, 'COST'),
 (1.0415131105322977, 0.36400286927796205, 'ABT'),
 (1.046658278251328, 0.9955638582968797, 'ACN'),
 (1.0987572408075144, 1.0660842732172688, 'MSFT'),
 (1.152816727036022, 0.35793174019028534, 'NEE'),
 (1.1976737036390077, 1.3367436601444522, 'GOOG'),
 (1.1998659276253096, 0.8655149801803501, 'CVS'),
 (1.2195931497649366, 0.8573309852082692, 'MO'),
 (1.2264110981153888, 1.0143275478309974, 'TGT'),
 (1.2565351315580555, 0.649599309935191, 'PM'),
 (1.2572305193191413, 0.41348223765196035, 'ABBV'),
 (1.263780051997507, 1.2541187384913304, 'UNP'),
 (1.2797325890784927, 1.3026896111532988, 'AAPL'),
 (1.3080834411931583, 1.058612109724245, 'UNH'),
 (1.3491543295133908, 1.1733291795709087, 'TXN'),
 (1.3514651550070829, 0.9429236073098203, 'NKE'),
 (1.3799141849863301, 0.7317051845797139, 'ORCL'),

In [17]:
threshold = 0
index = 0
while threshold < len(clean_stocks):
    if len(total_list) == 20:
        break
    beta_temp = get_beta_total(total_list)
    std_temp = get_average_std(total_list)
    beta_stock = list_of_beta[threshold][1]
    std_stock = list_of_beta[threshold][0]
    stock = list_of_beta[threshold][2]
#    print(std_temp,std_stock)
    if beta_stock <= beta_temp and std_stock < std_temp:
        total_list.append(stock)
        clean_stocks.remove(stock)
        list_of_beta.pop(threshold)
        threshold-=1
    threshold+=1
counting = 0
print(len(total_list))
while len(total_list) != 20:
    total_list.append(list_of_beta[0][2])
    list_of_beta.pop(0)
print(total_list)
        

14
['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO', 'KO', 'PEP', 'CL', 'COST', 'ABT', 'ACN', 'MSFT', 'NEE', 'GOOG', 'CVS']


In [18]:
final_returns = get_daily_returns(total_list)
final_returns.head()

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Date,,,,,,,,,,,,,,,,,,,,
2021-07-06,-1.045943,-0.448087,-0.739197,0.776798,0.058867,0.661052,-1.535144,-0.700057,-0.718435,0.178772,-0.553709,0.147743,0.330487,-0.020058,0.660795,0.183700,0.003608,1.144311,0.817286,-1.516439
2021-07-07,0.528496,0.735182,1.270234,0.000000,0.750114,1.218342,-1.449386,-0.759228,-0.310134,0.081111,0.816627,0.442553,0.841774,1.459163,0.883694,1.378522,0.817539,0.266215,0.236190,0.197094
2021-07-08,0.000000,-0.670234,-0.928338,0.195273,-0.014604,-0.981729,-0.556491,-0.710375,-0.809573,-0.226939,-0.349777,0.046739,-0.387135,0.610359,-0.508889,-0.148580,-0.896646,-0.265509,-0.692280,-1.376927
2021-07-09,0.901248,0.704755,0.839640,-0.882137,0.036499,1.275979,3.677358,2.751800,0.958130,-0.129972,0.609640,-0.253569,0.109306,1.282082,0.402476,1.122430,0.187444,-0.186339,0.307715,1.371226
2021-07-12,-0.949018,0.506239,-1.054519,0.382901,0.080277,0.842758,0.447218,-0.803443,0.105440,0.536845,0.036727,0.020062,0.606580,-1.088822,-0.776679,-0.335861,-0.223066,0.133346,0.763655,0.614853


In [19]:
correlation_values = pd.DataFrame(final_returns.corr())
correlation_values.head()

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Monthly Returns of CSCO,1.000000,0.431783,0.270445,0.153152,0.413716,0.332275,0.378010,0.273294,0.412120,0.229746,0.469199,0.375479,0.254952,0.427872,0.345192,0.584371,0.564646,0.190071,0.514603,0.378935
Monthly Returns of BMY,0.431783,1.000000,0.268171,0.032377,0.349729,0.173074,0.170115,0.051525,0.423604,0.273545,0.352593,0.338782,0.301257,0.223678,0.388126,0.420419,0.427358,0.241950,0.190694,0.491485
Monthly Returns of LMT,0.270445,0.268171,1.000000,-0.090981,0.044505,0.266573,0.261668,0.319884,0.323002,0.123065,0.294585,0.233500,0.104034,0.266454,0.047145,0.308594,0.193327,0.003775,0.144177,0.476443
Monthly Returns of MON,0.153152,0.032377,-0.090981,1.000000,0.026128,0.141571,0.013273,-0.041035,0.003940,-0.040426,0.045740,0.071291,-0.020418,0.067134,0.063011,0.119582,0.096793,0.015819,0.159905,0.010603
Monthly Returns of PG,0.413716,0.349729,0.044505,0.026128,1.000000,0.198295,0.074273,-0.002846,0.228960,0.464961,0.552413,0.631309,0.415836,0.323990,0.397196,0.385522,0.322459,0.355241,0.190129,0.253768


In [20]:
final_beta_std = []
for stock in total_list:
    final_beta_std.append((final_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
final_beta_std.sort()
final_beta_std

[(0.4310261262982297, -0.04813666256123079, 'MON'),
 (0.7601213194267271, 0.1386584965653855, 'PG'),
 (0.7688321597213277, 0.4999981723218368, 'KO'),
 (0.8070678497721255, 0.27208094687927487, 'PEP'),
 (0.8252266938315554, 0.21284068511016407, 'SO'),
 (0.8747186606103451, 0.7336717999040371, 'T'),
 (0.9473982893659325, 0.1479497575862508, 'CL'),
 (0.9523077719862846, 0.5814648713566358, 'COST'),
 (0.9653513769944979, 0.8127925379186896, 'LMT'),
 (1.0195849623485882, 0.4884316944235335, 'BMY'),
 (1.0335788002409754, 0.786460087670376, 'CSCO'),
 (1.0415131105322977, 0.36400286927796205, 'ABT'),
 (1.046658278251328, 0.9955638582968797, 'ACN'),
 (1.0987572408075144, 1.0660842732172688, 'MSFT'),
 (1.152816727036022, 0.35793174019028534, 'NEE'),
 (1.1976737036390077, 1.3367436601444522, 'GOOG'),
 (1.1998659276253096, 0.8655149801803501, 'CVS'),
 (1.2247162443231847, 0.8349647502245604, 'SBUX'),
 (1.636171983361825, 1.672490541016706, 'KMI'),
 (1.8787728953059228, 1.949598933240963, 'SPG')]

In [21]:
print(get_average_std(total_list))
print(get_beta_total(total_list))

1.04310800607395
0.703455399648219


In [22]:
def create_portfolio(stock_list,balance):
    ticker_frame = pd.DataFrame()
    for x in range (len(stock_list)):
        ticker_frame = pd.concat([ticker_frame, pd.DataFrame((hist_data['Close of '+stock_list[x]]*(1/len(stock_list)*balance)/hist_data['Close of '+stock_list[0]])).rename(columns={'Close of '+stock_list[x]:'Value of '+stock_list[x]})], axis=1))
    return ticker_frame

SyntaxError: unmatched ')' (<ipython-input-22-c8f125fe9765>, line 4)

In [ ]:
## Check the value of the portfolio with all 10,000 dollars invested in it
base_portfolio = create_portfolio(['AAPL'],100000)
base_portfolio['Balance'] = base_portfolio.sum(axis=1)
base_portfolio['Monthly Returns']= base_portfolio['Balance'].pct_change()* 100
## Calculate the sharpe ratio and assign it to a variable
base_value = base_portfolio.mean().loc['Monthly Returns']*base_portfolio.std(axis=0,skipna=True).loc['Monthly Returns']
## Place the sharpe ratio into a list
sharpe_list = [base_value]
## Iterate through weightings 1-100, and calculate the sharpe ratio
new_list = [stock_list[0]]
list_of_weights = [stock_list[0]]
weighted_portfolio = pd.DataFrame()
for x in range (min(len(cleaned_list),20)):
    overall_portfolio = create_portfolio(list_of_weights,new_list,100000)
    new_list.append(stock_list[x])
    for x in range(1,101):
        current_portfolio = pd.concat([create_monthly(overall_portfolio, (10000*x/100)), create_monthly(stock_list[x], (10000*(100-x)/100))],axis=1)
        current_portfolio['Balance'] = current_portfolio.sum(axis=1)
        current_portfolio['Monthly Returns']= current_portfolio['Balance'].pct_change()* 100
        current_value = current_portfolio.mean().loc['Monthly Returns']/current_portfolio.std(axis=0,skipna=True).loc['Monthly Returns']
        ## Add the sharpe ratio to a list
        sharpe_list.append(current_value)
        ## If the sharpe ratio is larger than the current largest one, replace it
        if base_value < current_value:
            ## Take the ideal weighting and store it in a variable
            weighting=x
            base_value = current_value
        

    

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.